# Introduction

This interactive Jupyter Notebook aims to shed light on the development process behind the following functions within the `queries` package, as well as to serve as additional documentation on their usage: 
* `get_cols_query` - returns an SQL query to get the column names of a table in a database
* `make_AV2017_pop_query` - returns an SQL query to construct a table of tumour data from the AV2017 snapshot    
* `make_totals_query` - returns an SQL query to get counts of values in a list of columns
* `all_counts_query` - returns an SQL query to get linked value counts from a list of columns in a pair of datasets.

The primary goal of these functions is to produce SQL code which constructs a table of value counts by field over two tables of data which share common data fields and categorical values, in order to compare the two tables. The latter two functions acheive this goal, whilst the former two functions are used in constructing one of the data tables for our use-case.

# Setup: Connecting to an SQL database with Python

In [ ]:
import pandas as pd
import getpass

from database import connect
import queries

In [ ]:
# Log into the database
db = connect(input('username:'), getpass.getpass('password:'))

# Extract relevant column names for constructing queries
First we get a list of column names from the simulated and real tumour tables. By requiring the number of distinct entries in the columns we select to be fewer than 100,000 we exclude index columns whilst retaining columns containing dates. This limit would need updating as timescales increase.

In [ ]:
col_condition = 'num_distinct < 100000'

col_names_sim = pd.read_sql_query(queries.get_cols_query('ANALYSISPAULCLARKE', 'SIM_AV_TUMOUR_FINAL', col_condition),
                                  db)['column_name']

col_names_av2017 = pd.read_sql_query(queries.get_cols_query('AV2017', 'AT_TUMOUR_ENGLAND', col_condition),
                                     db)['column_name']

col_names_av2017_exp = pd.read_sql_query(queries.get_cols_query('AV2017', 'AT_TUMOUR_EXPERIMENTAL_ENGLAND', col_condition),
                                         db)['column_name']

For each column in the simulated table, we want to find the corresponding table of real data which has that column in order to make comparisons.

At the time of writing, 21 out of the 31 column names in `SIM_AV_TUMOUR` are present in `AV2015.AV_TUMOUR`, which increases to 25 out of 31 in `AV2017.AT_TUMOUR_ENGLAND` with the addition of 4 Gleason grade columns (prostate cancer). 

In [ ]:
sim_cols_in_av2017 = col_names_sim.loc[col_names_sim.isin(col_names_av2017)]

An additional 5 columns, namely `['CANCERCAREPLANINTENT','PERFORMANCESTATUS','CNS','ACE27','DATE_FIRST_SURGERY']`, are covered by joining data from `AV2017.AT_TUMOUR_EXPERIMENTAL_ENGLAND` (along the `TUMOURID` column), bringing coverage up to 30 out of 31 simulated columns.

The column `GRADE` is present in both `AV2017.AT_TUMOUR_ENGLAND` and `AV2017.AT_TUMOUR_EXPERIMENTAL_ENGLAND`, and we specifically compare to `AV2017.AT_TUMOUR_ENGLAND.GRADE` since this was the column used in the original training dataset.

In [ ]:
sim_cols_in_av2017_exp = col_names_sim.loc[col_names_sim.isin(
    col_names_av2017_exp) & (col_names_sim != 'GRADE')]

Finally, the column `QUINTILE_2015` is obtained from the Index of Multiple Deprivation table `IMD.ID2015`  (by joining along area code `LSOA11_CODE`). 

In [ ]:
at_tumour_subq_cols = 'TUMOURID, LSOA11_CODE'
at_tumour_exp_subq_cols = 'TUMOURID'
all_cols_real = 'multi_depr_index.QUINTILE_2015'

for col_name in sim_cols_in_av2017:
    at_tumour_subq_cols += ', {}'.format(col_name)
    all_cols_real += ', at_tumour.{}'.format(col_name)

for col_name in sim_cols_in_av2017_exp:
    at_tumour_exp_subq_cols += ', {}'.format(col_name)
    all_cols_real += ', at_tumour_exp.{}'.format(col_name)

In [ ]:
print('Non-index columns in simulated table: \n', col_names_sim.to_list(), '\n')
print('Relevant column names in AV2017.AT_TUMOUR_ENGLAND: \n',
      at_tumour_subq_cols, '\n')
print('Relevant column names in AV2017.AT_TUMOUR_EXPERIMENTAL_ENGLAND: \n',
      at_tumour_exp_subq_cols, '\n')
print('Fields to be drawn from joined table of real data: \n', all_cols_real)

# Construct tables of real and simulated data
Here we construct the SQL queries to build the tables of real and simulated data, using the lists of column names from the previous section. Constructing the table of real data has now been hard coded into the function `queries.make_AV2017_pop_query`.

In [ ]:
sql_sim_pop = '''SELECT * FROM analysispaulclarke.sim_av_tumour_final'''

# sql_real_pop = queries.make_AV2017_pop_query()
sql_real_pop = '''SELECT {all_cols} FROM
(SELECT {at_tumour_cols} FROM AV2017.AT_TUMOUR_ENGLAND WHERE (diagnosisdatebest BETWEEN '01-JAN-2013' AND '31-DEC-2015') 
AND STATUSOFREGISTRATION = 'F' AND CTRY_CODE = 'E' AND DEDUP_FLAG = 1) at_tumour
LEFT JOIN 
(SELECT {at_tumour_exp_cols} 
FROM AV2017.AT_TUMOUR_EXPERIMENTAL_ENGLAND) at_tumour_exp
ON at_tumour.tumourid = at_tumour_exp.tumourid
LEFT JOIN IMD.ID2015 multi_depr_index
ON at_tumour.LSOA11_CODE = multi_depr_index.LSOA11_CODE
'''.replace('\n', ' ').format(all_cols=all_cols_real,
                              at_tumour_cols=at_tumour_subq_cols,
                              at_tumour_exp_cols=at_tumour_exp_subq_cols)

# Calculate value counts grouped by field for each table
Here we make two large SQL queries composed of smaller subqueries which collect category sizes for each category in each column, one for the simulated table and one for the real data.

In [ ]:
sql_sim_totals = queries.make_totals_query(sql_sim_pop, col_names_sim, 'sim')
sql_real_totals = queries.make_totals_query(
    sql_real_pop, col_names_sim, 'real')

In [ ]:
totals_sim = pd.read_sql_query(sql_sim_totals, db)
print(totals_sim.shape)

In [ ]:
totals_real = pd.read_sql_query(sql_real_totals, db)
print(totals_real.shape)

# Combine the two tables for easier comparison

Now we combine the results into a single table, joining on column name and value whilst accounting for the following special cases:
- Matching the counts of `NULL` values in each column
- Match `CREG_CODE` values, ignoring the first character, as simulated CREG codes are prefixed with 'L' rather than 'Y'
- Match `QUINTILE_2015` values using only the first character (an integer between 1 and 5 (inclusive)), ignoring any descriptive text to avoid issues with upper/lower case
- Filling null counts (which could not be matched between tables) with 0

Constructing this combined table of value counts has now been refactored into the function `queries.all_counts_query`.

In [ ]:
# sql_combined_totals = all_counts_query(sql_sim_pop, sql_real_pop, col_names_sim)
sql_combined_totals = '''WITH population_real AS ({real_pop_query}),
population_sim AS ({sim_pop_query}),
r AS ({real_totals_query}),
s AS ({sim_totals_query})
SELECT
NVL(r.column_name, s.column_name) AS col_name,
NVL(r.val, s.val) AS val,
NVL(counts_real, 0) AS counts_r,
NVL(counts_sim, 0) AS counts_s
FROM r FULL OUTER JOIN s
ON (r.column_name = s.column_name AND (r.val = s.val OR (r.val IS NULL AND s.val IS NULL)))
OR (r.column_name = 'CREG_CODE' AND s.column_name = 'CREG_CODE' AND SUBSTR(r.val, 2) = SUBSTR(s.val, 2))
OR (r.column_name = 'QUINTILE_2015' AND s.column_name = 'QUINTILE_2015' AND SUBSTR(r.val, 1, 1) = SUBSTR(s.val, 1, 1))
'''.replace('\n', ' ').format(real_pop_query=sql_real_pop,
                              sim_pop_query=sql_sim_pop,
                              real_totals_query=queries.make_totals_query(
                                  sql_real_pop, col_names_sim, 'real', standalone=False),
                              sim_totals_query=queries.make_totals_query(sql_sim_pop, col_names_sim, 'sim', standalone=False))

In [ ]:
totals_comb = pd.read_sql_query(sql_combined_totals, db)
print(totals_comb.shape)

In [ ]:
totals_comb.head(20)